In [2]:
# 크롬 드라이버 세팅
chrome_driver = '../chromedriver' # 파일 확장자 이름 미표기

In [3]:
# 수집 앱 URL
URL = 'https://play.google.com/store/apps/details?id=com.dearu.bubble.playm&hl=ko&gl=US'

In [4]:
# 패키지 import
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
import random
import pandas as pd

In [5]:
# 무한 스크롤 함수
def scroll(modal):
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
        while True:
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
                
    except Exception as e:
        print("에러 발생: ", e)

In [6]:
# 데이터 크롤링
# 크롬 드라이버 세팅
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(chrome_driver, options = chrome_options)
# 페이지 열기
driver.get(URL)
# 페이지 로딩 대기
wait = WebDriverWait(driver, 5)

# '리뷰 모두 보기' 버튼 렌더링 확인(path 수정 @2022-06-22)
all_review_button_xpath = '/html/body/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/div/div/div[5]/div/div/button/span'
button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_button_xpath)))
# '리뷰 모두 보기' 버튼 클릭
driver.find_element(By.XPATH,all_review_button_xpath).click()

# '리뷰 모두 보기' 페이지 렌더링 대기
all_review_page_xpath = '/html/body/div[4]/div[2]/div/div/div/div/div[2]'
page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_page_xpath)))

# 페이지 무한 스크롤 다운
modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
scroll(modal)

C:\Users\Home\AppData\Local\Temp\ipykernel_19924\1532846636.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver, options = chrome_options)


스크롤 완료


In [7]:
# html parsing하기
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

In [8]:
# 크롬 드라이버 강제종료
driver.quit()

In [9]:
# html 데이터 저장
with open("../dataset/data_html[IST].html", "w", encoding = 'utf-8') as file:
    file.write(str(soup_source))

In [10]:
# 데이터 프레임 변환
# 리뷰 데이터 클래스 접근
review_source = soup_source.find_all(class_ = 'RHo1pe')
# 리뷰 데이터 저장용 배열
dataset = []
# 데이터 넘버링을 위한 변수
review_num = 0 
# 리뷰 1개씩 접근해 정보 추출
for review in review_source:
    review_num+=1
    # 리뷰 등록일 데이터 추출
    date_full = review.find_all(class_ = 'bp9Aid')[0].text
    date_year = date_full[0:4] # 연도 데이터 추출
    # 해당 단어가 등장한 인덱스 추출
    year_index = date_full.find('년')
    month_index = date_full.find('월')
    day_index = date_full.find('일')
    
    date_month = str(int(date_full[year_index+1:month_index])) # 월(Month) 데이터 추출
    # 월 정보가 1자리의 경우 앞에 0 붙이기(e.g., 1월 -> 01월)
    if len(date_month) == 1:
        date_month = '0' + date_month
    
    date_day = str(int(date_full[month_index+1:day_index])) # 일(Day) 데이터 추출 
    # 일 정보가 1자리의 경우 앞에 0 붙여줌(e.g., 7일 -> 07일)
    if len(date_day) == 1:
        date_day = '0' + date_day
    
    # 리뷰 등록일 full version은 최종적으로 yyyymmdd 형태로 저장
    date_full = date_year + date_month + date_day
    user_name = review.find_all(class_ = 'X5PpBb')[0].text # 닉네임 데이터 추출
    rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10] # 평점 데이터 추출
    content = review.find_all(class_ = 'h3YV2d')[0].text # 리뷰 데이터 추출

    data = {
        "id": review_num, 
        "date": date_full,
        "dateYear": date_year,
        "dateMonth": date_month,
        "dateDay": date_day,
        "rating": rating,
        "userName": user_name,
        "content": content
    }
    dataset.append(data)

In [11]:
# 데이터프레임 저장
df = pd.DataFrame(dataset)
df.to_csv('../dataset/review_dataset[IST].csv', encoding = 'utf-8-sig') # csv 파일로 저장

In [12]:
# 저장한 리뷰 정보 불러오기
df = pd.read_csv('../dataset/review_dataset[IST].csv', encoding = 'utf-8-sig')
df = df.drop(['Unnamed: 0'], axis = 1) # 불필요한 칼럼 삭제
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
0,1,20220829,2022,8,29,3,별하,"리뷰를 삭제해버려서 다시 씁니다. 버블의 최대 장점: 나의 아이돌, 나의 최애와 대..."
1,2,20220829,2022,8,29,4,나연,제가 좋아하는 최애랑 연락할수 있어서 좋아요~좋은데 어쩔때마다 보내면 자꾸 비행기 ...
2,3,20220307,2022,3,7,1,엔비,업데이트 최신버전이고 지웠다 다시 설치하길 반복해도 여전히 발송이 안됩니다. 데이터...
3,4,20211105,2021,11,5,4,미나,은지는 너무 귀여워서 좋은데 안드로이드는 캡쳐 막고 아이폰은 가능한 시스템은 도대체...
4,5,20210714,2021,7,14,3,빅토니네강댕댕,캡쳐 제한 됫던데 왜 애플기기는 되요? 안되게 할거면 다 안되게 해야지 갤럭시유저 ...
5,6,20211214,2021,12,14,5,당근,"진짜 갤럭시도 캡쳐기능 주세요,,...ㅠ 우리언니 예쁜말 저장해서 두고두고 보고싶단..."
6,7,20211108,2021,11,8,4,박수빈,지문인식 기능도 만들어주세요!!다 좋은데 지문인식기능이 있었으면 좋겟습니다!!^.^
7,8,20220127,2022,1,27,2,ᄋᄋ,갤럭시도 캡쳐기능이랑 음성 메세지 영상으로 저장되게 해주세요ㅜㅜ 유출 안 할게요 제...
8,9,20220526,2022,5,26,4,დ,다 좋긴한데요 멤버가 여러명 있다보니까 다 따로와서 하나하나 확인하고 답장하기가 힘...
9,10,20210515,2021,5,15,5,VICYA앨냐,자주 와줘서 돈 아깝다는 생각 정말 1도 안하게 잘 쓰구 있습니다아♡
